### Week 3 Kaggle Inclass Competitions

# Анализ тональности отзывов

In [1]:
# импортируем необходимые модули
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics

import sys
import warnings
# Disabling warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
%matplotlib inline

In [2]:
# считываем обучающую и тестовую выборку
train_data = pd.read_csv('products_sentiment_train.tsv', sep='\t', header=None)
train_data.columns = ['text', 'label']
train_data.head()

text  label
0          2 . take around 10,000 640x480 pictures .      1
1  i downloaded a trial version of computer assoc...      1
2  the wrt54g plus the hga7t is a perfect solutio...      1
3  i dont especially like how music files are uns...      0
4  i was using the cheapie pail ... and it worked...      1

In [3]:
test_data = pd.read_csv('products_sentiment_test.tsv', sep='\t')
test_data.head()

Id                                               text
0   0  so , why the small digital elph , rather than ...
1   1  3/4 way through the first disk we played on it...
2   2  better for the zen micro is outlook compatibil...
3   3    6 . play gameboy color games on it with goboy .
4   4  likewise , i 've heard norton 2004 professiona...

In [4]:
# записываем тексты и метки в переменные
texts = train_data.text
labels = train_data.label

In [5]:
# проверим соотношение классов в обучающей выборке.
labels.value_counts() / len(labels)

# классы не сбалансированы, нужно будет это учесть в параметрах классификатора
# или настраивать порог отнесения к тому или иному классу 

1    0.637
0    0.363
Name: label, dtype: float64

In [6]:
# разделим обучающую выборку на тренировочную и валидационную часть (в принципе этого можно было пока не делать,
# для получения первого приближения можно было провести кросс-валидацию на всей обучающей выборке)
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.2,
                                                                      shuffle=True, stratify=labels, random_state=0)

In [7]:
# пишем простую функцию для первичного анализа векторайзеров и классификаторов на кросс-валидации
def text_classifier(vectorizer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("classifier", classifier)]
        )

In [8]:
# попробуем получить первое приближение на нескольких классификаторах и двух векторайзерах

for clf in [SVC, LogisticRegression, LinearSVC, SGDClassifier, MultinomialNB,
            RandomForestClassifier, GradientBoostingClassifier]:
    for vect in [CountVectorizer, TfidfVectorizer]:
        print(clf, vect)
        print(cross_val_score(text_classifier(vect(), clf()), texts_train, labels_train, cv=10).mean())
        print("\n")

<class 'sklearn.svm._classes.SVC'> <class 'sklearn.feature_extraction.text.CountVectorizer'>
0.748125


<class 'sklearn.svm._classes.SVC'> <class 'sklearn.feature_extraction.text.TfidfVectorizer'>
0.764375


<class 'sklearn.linear_model._logistic.LogisticRegression'> <class 'sklearn.feature_extraction.text.CountVectorizer'>
0.7568750000000001


<class 'sklearn.linear_model._logistic.LogisticRegression'> <class 'sklearn.feature_extraction.text.TfidfVectorizer'>
0.764375


<class 'sklearn.svm._classes.LinearSVC'> <class 'sklearn.feature_extraction.text.CountVectorizer'>
0.7550000000000001


<class 'sklearn.svm._classes.LinearSVC'> <class 'sklearn.feature_extraction.text.TfidfVectorizer'>
0.7681250000000001


<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'> <class 'sklearn.feature_extraction.text.CountVectorizer'>
0.748125


<class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'> <class 'sklearn.feature_extraction.text.TfidfVectorizer'>
0.75375


<class 'sklearn

## Первый способ. 

#### Позволяет занять 55-74 место на лидерборде.

In [9]:
# наилучшие результаты дает сочетание Tfidf c LinearSVC, SVC и LogisticRegression, а также CountVectorizer c наивным Байесом
# при дальнейшей настройке параметров векторайзеров и классификаторов сильно улучшить качество в случае наивного Байеса
# не удалось, Tfidf + LogisticRegression также проигрывали по качеству LinearSVC и SVC. 
# в итоге остановил выбор на 2 моделях: Tfidf + LinearSVC и Tfidf + SVC. Их и попробуем настроить.

# в параметрах векторайзера: 
# 1) пробовал различные аналайзеры (по словам и буквам), наилучший результат на валидационной части и на кросс-валидации
# достигался при использовании буквенных грам 
# 2) менял число букв - лучший результат получал на грамах от 3 до 13 символов.
# 3) использование стоп-слов встроенных в sklearn и nltk -  только ухудшало результаты.
# 4) пробовал добавлять стеммер - результат получался чуть хуже даже при использовании анализа по словам, а не по буквам.
# 5) установка максимальной и минимальной частоты (min_df и max_df) неожиданно не дала улучшения результата.

vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3,13))
X_train = vectorizer.fit_transform(texts_train)
X_test = vectorizer.transform(texts_test)

In [10]:
# в настройках классификатора:
# 1) использовал различные ядра - лучший результат дает сигмоид.
# 2) пробовал различные соотношение весов классов - лучший результат дает дефолтный 'balanced'.
# 3) пытался самостоятельно относить объекты к классам, используя вероятности (чтобы SVC выдавал вероятности нужно 
#    probability=True), самостоятельно подбирая порог отнесения к тому или иному классу.
#    Толкового ничего не нашел: при автоматической балансировке классов в параметрах классификатора, наилучший разделяющий 
#    порог был практически равен 0.5. Если не производить балансировку, то лучший порог смещался в район 0.365 (т.е. примерно
#    соответствовал доле класса 0 в обучающей выборке.) 
clf = SVC(kernel='sigmoid', class_weight='balanced', probability=False)
clf.fit(X_train, labels_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
# проверим точность на валидационной части обучающей выборки
metrics.accuracy_score(labels_test, clf.predict(X_test))

0.8125

In [12]:
# использовал код ниже для получения вероятностей отнесения к тому или иному классу
# ВНИМАНИЕ долго считает!

# res = pd.DataFrame(labels_test)
# res['prob_0'] = 0 
# res['prob_1'] = 0
# res['y'] = 0
# res.reset_index(inplace=True)
# for i in range(res.shape[0]): 
#     res.loc[i, 'prob_0'] = clf.predict_proba(X_test)[i][0]
#     res.loc[i, 'prob_1'] = clf.predict_proba(X_test)[i][1]

In [13]:
# код ниже рисует график точности при различном пороге.

# thresh_list = [0.002 * i for i in range(100, 300)]
# accuracy = []
# for thresh in thresh_list:
#     res['y'] = 0
#     res.loc[res['prob_1'] > thresh, 'y'] = 1
#     accuracy.append((res.label == res.y).sum() / res.shape[0])
# plt.figure(figsize=(15,10))
# plt.grid()
# plt.plot(thresh_list, accuracy)

In [14]:
# обучаем векторайзер и классификатор на всей обучающей выборке
vectorizer_full = TfidfVectorizer(analyzer='char', ngram_range=(3,13))
X_full = vectorizer_full.fit_transform(texts)
X_full_test = vectorizer_full.transform(test_data.text)

In [15]:
clf_full = SVC(kernel='sigmoid', class_weight='balanced', probability=False)
clf_full.fit(X_full, labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
# после настройки параметров на кросс-валидации получаем точность выше 80% на всей обучающей выборке.
cross_val_score(clf_full, X_full, labels, cv=10).mean()

0.8045000000000002

In [17]:
test_data['y'] = clf_full.predict(X_full_test)

In [18]:
pd.set_option('max_colwidth', 300)
test_data[:5]

Id  \
0   0   
1   1   
2   2   
3   3   
4   4   

                                                                                                                                                                                           text  \
0  so , why the small digital elph , rather than one of the other cameras with better resolution or picture quality ? size [ + 2 ] # # because , unless it 's small , i won 't cary it around .   
1                                                                                3/4 way through the first disk we played on it ( naturally on 31 days after purchase ) the dvd player froze .    
2                                                                                                                                           better for the zen micro is outlook compatibility .   
3                                                                                                                                               6 . play gameboy color games on it with goboy .   
4                                                                                                                         likewise , i 've heard norton 2004 professional version is fine too .   

   y  
0  1  
1  0  
2  1  
3  1  
4  0

In [19]:
test_data.y.sum()

317

In [20]:
test_data[['Id','y']].to_csv('submission_SVC.csv', index = False)

In [21]:
# проверяем корректность файла для сабмита.
# Данный сабмит дает с 55 по 66 место на лидерборде.
SVC_subm = pd.read_csv('submission_SVC.csv')
SVC_subm

Id  y
0      0  1
1      1  0
2      2  1
3      3  1
4      4  0
..   ... ..
495  495  0
496  496  1
497  497  0
498  498  0
499  499  1

[500 rows x 2 columns]

## Второй способ. 

#### Также позволяет занять 55-74 место на лидерборде.

Подробно расписывать не буду, смысл тот же - настройка векторайзера (Tfidf) и классификатора (LinearSVC).
LinearSVC не прогнозирует вероятности, поэтому с порогом отнесения к тому или иному классу не игрался.

In [22]:
vectorizer2 = TfidfVectorizer(analyzer='char', ngram_range=(3,13))
X_train2 = vectorizer2.fit_transform(texts)
X_test2 = vectorizer2.transform(test_data.text)

In [23]:
# for c in [0.05 * i for i in range(11, 17)]:
#     for cl_weight in [0.005 * j for j in range(72, 76)]:
#         clf2 = LinearSVC(C=c, penalty='l2', class_weight={1: cl_weight, 0: 1-cl_weight})
#         clf2.fit(X_train2, labels)
#         print(c, cl_weight, cross_val_score(clf2, X_train2, labels, cv=10).mean())

In [24]:
clf2 = LinearSVC(C=0.65, penalty='l2', class_weight={1: 0.365, 0: 1-0.365})
clf2.fit(X_train2, labels)

LinearSVC(C=0.65, class_weight={0: 0.635, 1: 0.365}, dual=True,
          fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
          max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
          tol=0.0001, verbose=0)

In [25]:
test_data['y2'] = clf2.predict(X_test2)
pd.set_option('max_colwidth', 300)
test_data[:10]

Id  \
0   0   
1   1   
2   2   
3   3   
4   4   
5   5   
6   6   
7   7   
8   8   
9   9   

                                                                                                                                                                                                                            text  \
0                                   so , why the small digital elph , rather than one of the other cameras with better resolution or picture quality ? size [ + 2 ] # # because , unless it 's small , i won 't cary it around .   
1                                                                                                                 3/4 way through the first disk we played on it ( naturally on 31 days after purchase ) the dvd player froze .    
2                                                                                                                                                                            better for the zen micro is outlook compatibility .   
3                                                                                                                                                                                6 . play gameboy color games on it with goboy .   
4                                                                                                                                                          likewise , i 've heard norton 2004 professional version is fine too .   
5                                                                                                                                                       mine was 2 weeks old and i chucked it in the trash , where it belongs .    
6                   i find it very stable and comfortable to use , it doesn 't jump around or make me feel like i need to use a lot of arm strength to keep it under control , and it has very good visibility to see your cut .   
7  styling / ergonomics : the keys are small , which generally doesn 't bother me if they are spaced because i can employ a little ' braille , ' but the keys run into each other , so careful attn. must be paid when dialing .   
8                                                                                         at first i thought it is only a isolated incident , but i was proven wrong when the player would not read my " alias " dvds as well .    
9                                                                                                                                                                                                      - light , compact design    

   y  y2  
0  1   1  
1  0   0  
2  1   1  
3  1   1  
4  0   0  
5  0   0  
6  1   1  
7  0   0  
8  0   0  
9  1   1

In [26]:
test_data[['Id','y2']].to_csv('submission_LinearSVC.csv', index = False)

In [27]:
subm_linearSVC = pd.read_csv('submission_LinearSVC.csv')
subm_linearSVC.columns = ['Id', 'y']
subm_linearSVC.to_csv('submission_LinearSVC.csv', index = False)

#### Сочетание 1 и 2 способа + ручой труд
Посмотрим на позиции, где классификаторы из первого и второго способа разошлись в своих оценках. 

In [28]:
test_data['y3'] = (test_data['y'] + test_data['y2']) / 2
# test_data[test_data['y'] != test_data['y2']]
test_data[test_data['y3'] == 0.5]

Id  \
29    29   
49    49   
99    99   
124  124   
162  162   
200  200   
203  203   
234  234   
243  243   
307  307   
309  309   
340  340   
352  352   
367  367   
372  372   
407  407   
441  441   
495  495   

                                                                                                                                                                                                                         text  \
29                                                                                                                                             the champ did not start letting a little odor through until well over a year .   
49                                                                                                                                           for those of us that like to tinker , i think creative offers some advantages .    
99                                                                                                                                                                     there is a clock , and date , which can get helpful .    
124                                                                                          naturally i did quite a bit of review before purchasing ( i recomend www.nomadness.net ) , and price-wise zens are unbeatable .    
162                                                                                                                                                                     finally , i gave up and i purchased a router razier .   
200                                                                                                                                                   the stand that comes with it is very useful to prop it up on the desk .   
203                                                                                                        another thing you should know , the metal on the flip side of the ipod is very tacky , and scratches very easily .   
234                                                                                                                                                    also the battery life isn 't great but it 's sufficient for my needs .   
243                                                                                                                                                                            the player itself has all sorts of problems .    
307                                                                                                                                                pros : large hard drive for the $ $ -- 20gb and 40gb are both affordable .   
309                                                                                                                           the other thing i do n't like about this 2600 is that it makes the color look over saturated .    
340                                                                                                          for me , it is as important that the tool be comfortable to use as it is that it produce a high-quality result .   
352                                                                                                                                                                                i 've had no problems with the software .    
367                                                                                                                                                           power for the tech-savvy , ease of use for the less fortunate .   
372  we can start by losing the shiny miracle-material cover that you can literally scratch just by looking at it wrong , and thus eliminate the need for third-party " iskin " protective covers for an extra thirty bucks .   
407                                                                                                                                                you can take pictures seconds 

По идее, можно было бы смотреть вероятности, которые выдают классификаторы на данных объектах и усреднять их, но LinearSVC
не выдает вероятности.
Можно было бы попробовать добавить какой-нибудь третий классификатор, но в данной задаче спорных позиций не много.
Поэтому решил попробовать вручную определить метки (по некоторым позициям это было не так просто :) ) и посмотреть на результат.

In [29]:
test_data.loc[29, 'y3'] = 1
test_data.loc[49, 'y3'] = 1
test_data.loc[99, 'y3'] = 1
test_data.loc[124, 'y3'] = 1
test_data.loc[162, 'y3'] = 0
test_data.loc[200, 'y3'] = 1
test_data.loc[203, 'y3'] = 0
test_data.loc[234, 'y3'] = 0
test_data.loc[243, 'y3'] = 0
test_data.loc[307, 'y3'] = 1
test_data.loc[309, 'y3'] = 0
test_data.loc[340, 'y3'] = 1
test_data.loc[352, 'y3'] = 1
test_data.loc[367, 'y3'] = 1
test_data.loc[372, 'y3'] = 0
test_data.loc[407, 'y3'] = 0
test_data.loc[441, 'y3'] = 0
test_data.loc[495, 'y3'] = 0


In [30]:
test_data['y3'] = test_data['y3'].astype('int64')
test_data

Id  \
0      0   
1      1   
2      2   
3      3   
4      4   
..   ...   
495  495   
496  496   
497  497   
498  498   
499  499   

                                                                                                                                                                                             text  \
0    so , why the small digital elph , rather than one of the other cameras with better resolution or picture quality ? size [ + 2 ] # # because , unless it 's small , i won 't cary it around .   
1                                                                                  3/4 way through the first disk we played on it ( naturally on 31 days after purchase ) the dvd player froze .    
2                                                                                                                                             better for the zen micro is outlook compatibility .   
3                                                                                                                                                 6 . play gameboy color games on it with goboy .   
4                                                                                                                           likewise , i 've heard norton 2004 professional version is fine too .   
..                                                                                                                                                                                            ...   
495                                                                                     i took perfect care of this player and still it died in the week after the lousy 90 day warranty ended .    
496                                                                                                                                                             it 's a very intuitive program .    
497                                                                                                                        the only drawback is the viewfinder is slightly blocked by the lens .    
498                                                                                                                                              it films 10 second video , for crying out loud .   
499                                                                                                                                                                everything shines of quality .   

     y  y2  y3  
0    1   1   1  
1    0   0   0  
2    1   1   1  
3    1   1   1  
4    0   0   0  
..  ..  ..  ..  
495  0   1   0  
496  1   1   1  
497  0   0   0  
498  0   0   0  
499  1   1   1  

[500 rows x 5 columns]

In [31]:
test_data[['Id','y3']].to_csv('submission_final.csv', index = False)

In [32]:
# данный сабмит с частичным использованием ручного труда дает 15 место на лидерборде.
# Если будет время, попробую уйти от использования рук на спорных объектах.
submission_final = pd.read_csv('submission_final.csv')
submission_final.columns = ['Id', 'y']
submission_final.to_csv('submission_final.csv', index = False)